In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import numpy as np
import onnx
import onnxruntime

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [5]:
def train( model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 12000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [7]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(42)


kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                  transform=transforms.Compose([
                  transforms.RandomAffine(
                  degrees=30, translate=(0.5, 0.5), scale=(0.25, 1),
                  shear=(-30, 30, -30, 30)),
                  transforms.ToTensor(),
                  transforms.Normalize((0.1307,), (0.3081,))
            ])), batch_size=32, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])),  batch_size=32, shuffle=True, **kwargs)

In [16]:
model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1.0)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
for epoch in range(1, 71):
    train( model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299779

Test set: Average loss: 1.6225, Accuracy: 4510/10000 (45%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.031099

Test set: Average loss: 1.0097, Accuracy: 7143/10000 (71%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.984479

Test set: Average loss: 0.8686, Accuracy: 7108/10000 (71%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.745221

Test set: Average loss: 0.6131, Accuracy: 8409/10000 (84%)

Train Epoch: 5 [0/60000 (0%)]	Loss: 1.719652

Test set: Average loss: 0.5608, Accuracy: 8632/10000 (86%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 1.730396

Test set: Average loss: 0.4884, Accuracy: 8770/10000 (88%)

Train Epoch: 7 [0/60000 (0%)]	Loss: 0.972951

Test set: Average loss: 0.4832, Accuracy: 8751/10000 (88%)

Train Epoch: 8 [0/60000 (0%)]	Loss: 1.494474

Test set: Average loss: 0.4967, Accuracy: 8683/10000 (87%)

Train Epoch: 9 [0/60000 (0%)]	Loss: 1.475579

Test set: Average loss: 0.4715, Accuracy: 8713/10000 (87%)

Train Epoch: 10 [0/60000 (0%)]	Loss: 1.679146


In [18]:
torch.save(model.state_dict(), "pytorch_model.pt")

In [30]:
dummy_input = torch.randn(1,1,28,28).to(device)
torch.onnx.export(model, dummy_input, 'onnx_model.onnx', verbose=True)

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1345: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [32]:
session = onnxruntime.InferenceSession('./onnx_model.onnx')

In [33]:
input_name = session.get_inputs()[0].name
print(input_name)
output_name = session.get_outputs()[0].name
print(output_name)

input.1
17


In [34]:
session.get_inputs()[0].shape

[1, 1, 28, 28]

In [38]:
for i in test_loader:
    sample = i
    break

In [63]:
x = sample[0][0].numpy()
x

array([[[-0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, -0.42421296, -0.42421296, -0.42421296],
        [-0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, -0.42421296, -0.42421296, -0.42421296],
        [-0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, -0.42421296, -0.42421296, -0.42421296,
         -0.42421296, 

In [64]:
y = sample[1][0]
y

tensor(8)

In [65]:
x.shape

(1, 28, 28)

In [66]:
x = np.expand_dims(x, axis=0)
x.shape

(1, 1, 28, 28)

In [68]:
out = session.run([output_name], {input_name: x})

In [69]:
np.argmax(out)

8